## *Import tools*

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from keras.layers import (Dropout, Input, Dense, Conv2D, MaxPooling2D, 
                          GlobalAveragePooling2D, UpSampling2D, Conv2DTranspose, 
                          Reshape, Flatten, Activation, BatchNormalization)
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.layers.advanced_activations import LeakyReLU
from keras.initializers import RandomNormal
from keras.optimizers import Adam

## *Get the data*

In [ ]:
data = pd.read_csv("path-to-dataset").astype('float32')
data.head()

## *Data preprocessing*

In [ ]:
width, height, channel = 28, 28, 1 # 28*28  grayscale images

In [ ]:
X = data.iloc[:,1:].values
X = X.reshape((len(X), width, height))
np.random.shuffle(X)
X.shape

## *Normalize the data*

In [ ]:
X = (X - 127.5) / 127.5 # between [-1, 1]

## *Visualize some alphabets*

In [ ]:
def show_data(X, title=""):
    plt.figure(figsize=(11,11))
    
    i = 1
    for img in X:
        plt.subplot(10, 10, i)
        plt.imshow(img.reshape((height, width)), cmap='gray')
        plt.axis('off')
        i+=1
        if i>100: break
    
    plt.suptitle(title, fontsize = 25)
    plt.show()
    
show_data(X, title="Original Alphabets")

## *Customize the Adam optimizer*

In [ ]:
mod_adam = Adam(0.0002, 0.5)

## *Set parameters for noise*

In [ ]:
noise_dim = 100 # most research papers suggest 100!
fixed_noise = np.random.normal(0, 1, size=(100, noise_dim)) # use this for visualizing model outputs

## *Build the Generator*

In [ ]:
def buildGenerator():
    model = Sequential()

    model.add(Dense(1024, input_dim=noise_dim))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    model.add(Dense(6272, input_dim=noise_dim))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    model.add(Reshape((7, 7, 128)))
    
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(64, (2, 2), padding='same', 
                     kernel_initializer=RandomNormal(0, 0.02)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(0.2))
    
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(channel, (3, 3), padding='same', activation = "tanh", 
                     kernel_initializer=RandomNormal(0, 0.02)))
    
    return model
    

In [ ]:
generator = buildGenerator()
generator.summary()

## *Build the Discriminator*

In [ ]:
def buildDiscriminator():
    model = Sequential()
    
    model.add(Conv2D(64, (5, 5), strides=2, padding='same', 
                     kernel_initializer=RandomNormal(0, 0.02), 
                     input_shape=(width, height, channel)))
    model.add(LeakyReLU(0.2))
    
    model.add(Conv2D(128, (5, 5), strides=2, 
                     kernel_initializer=RandomNormal(0, 0.02)))
    model.add(LeakyReLU(0.2))
    
    model.add(Flatten())
    
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=mod_adam)
    return model

In [ ]:
discriminator = buildDiscriminator()
discriminator.summary()

## *Combine Generator & Discriminator*

In [ ]:
noise = Input(shape=(noise_dim,))
fake_data = generator(noise)
discriminator.trainable = False
output = discriminator(fake_data)
gan = Model(noise, output)
gan.compile(loss='binary_crossentropy', optimizer=mod_adam)

## *For Visualizing model outputs*

In [ ]:
def show_generated_alphabets(title):
    imgs = generator.predict(fixed_noise)
    imgs = 0.5 * imgs + 0.5
    plt.figure(figsize=(11,11))
    
    i = 1
    for img in imgs:
        plt.subplot(10, 10, i)
        plt.imshow(img.reshape((height,width)), cmap='gray')
        plt.axis('off')
        i+=1
    plt.suptitle(title, fontsize = 25)
    plt.show()

## *Train the GAN*

In [ ]:
epochs = 101
batch_size = 128
steps_per_epoch = len(X)//batch_size

In [ ]:
for epoch in range(epochs):
    for batch in range(steps_per_epoch):
        input_gen = np.random.normal(0, 1, size=(batch_size, noise_dim))
        fake_data = generator.predict(input_gen)
        
        real_data = X[np.random.randint(0, X.shape[0], size=batch_size)]
        real_data = real_data.reshape((batch_size, width, height, channel))
        
        input_disc = np.concatenate((real_data, fake_data))

        label_disc = np.zeros(2*batch_size)
        label_disc[:batch_size] = 0.9
        loss_disc = discriminator.train_on_batch(input_disc, label_disc)

        label_gen = np.ones(batch_size)
        loss_gen = gan.train_on_batch(input_gen, label_gen)

    print("epoch: ", epoch)
    print("-"*80)
    print("discriminator loss: ", loss_disc)
    print("generator loss: ", loss_gen)
    
    if epoch % 5 == 0:
        show_generated_alphabets("Generated Alphabets")